### 1、对象存储

* 需求：项目中需要存储图片数据，存储可能需要使用文件系统服务器；
* 图片数据的存储和传输，对一般web网站来说，流量主要集中在图片上。
  * 1、可以自己手动搭建文件系统服务器，fastDFS，没有CDN内容分发；
  * 2、选择第三方服务器，七牛云已经集成了CDN，让用户访问图片数据的时候，数据距离用户越近越好。

* 使用流程：
  * 1、登录七牛云网站，选择对象存储
  * 2、安装七牛云pip install qiniu，新建存储空间，保存空间名称`sy28`，空间域名：`q0bhi0abu.bkt.clouddn.com`
  * 3、下载七牛云官方示例代码，进行代码封装，保存配置信息ak、sk
  * 4、修改token的有效期，修改上传文件的功能函数put_data(token,key,图片二进制数据)
  * 5、上传成功后，返回图片地址`http://q0bhi0abu.bkt.clouddn.com/FliqfxThWU9vPTueT_ifIdco_1HK`

### 2、上传头像

* 接口设计：四个点

~~~python
请求地址：‘/v1_0/user/photo’

请求方式：
POST表示新建/不合适
PUT表示更新/不建议
PATCH表示更新，建议使用
PUT方法要求，修改数据，需要传入所有相关参数，比如，修改头像，user_id/mobile/name/photo等
{
  ‘user_id’:xxx,
  'mobile':xxx,
  'name':xxx,
  'introduction':xxx,
  'photo':图片文件,
  ....
}

PATCH方法要求，修改数据，进需要传入相关参数，比如，修改头像，photo
{
  'photo':图片文件
}

请求参数：
参数名				是否必须			参数类型		参数位置		参数说明
photo					True				file			body 		用户选择的头像图片文件
user_id				True				int				headers 用户的身份信息	

返回数据
正常情况下：
return {'message':'ok','data':{'photo_url':'七牛云的空间域名+图片文件名'}}
异常情况下：
return {'message':'各种错误信息'}


~~~



* 需要异常处理的代码：

  * 1、网络io，上传下载、发送请求
  * 2、磁盘io，读写文件、查询数据
  * 3、类型转换；

  ~~~python
  a = '1'
  b = int(a)
  # 不能转换，需要异常处理
  a = 'a'
  b = int(a)
  
  
  ~~~



### 3、CDN

* 内容分发网络：用户距离资源越近越好。



### 4、缓存：

* 缓存的架构：临时存储，不能把缓存等价于redis；
* 头条项目中应用：
  * 1、sqlalchemy本地缓存
  * 2、redis的一级缓存
  * 3、7000~7005redis分布式集群(缓存数据，有有效期)，
  * 4、6380~6381redis的主从复制集（持久存储，没有有效期）

* 缓存数据：redis中能存什么？字符串、列表、哈希、集合、有序集合

  * 1、key、value形式的数据，例如：验证码、session信息等；
  * 2、视图通过查询mysql数据的结果，列表形式的记录，例如：浏览记录；
  * 3、存储一个页面、或是图片数据，例如：首页商品数据
  * 4、存储一个查询语句和结果，select xxx 哈希计算 = 字符串，key = 字符串，value = 查询结果

  

* redis数据类型的选择？字符串、列表、哈希、集合、有序集合

  * 需求：美多商城首页商品数据存储设计？？集合(去重、存的多)，建议哈希存储，可以维护单独的key、value数据；
  * 1、基本数据类型：字符串，单独的key、value，表示一条记录；节省空间；
  * 2、复合数据类型：除字符串外，占用空间较大，查询效率较高，多条key、value存储是一条数据；



### 5、缓存有效期和缓存淘汰

* redis中设置有效期的作用：
  * 1、清理内存空间；
  * 2、达到数据的弱一致性，在有效期过了以后，redis会和mysql数据同步；

* redis中数据的过期策略：

  * 需求：redis中设置数据有效期为10秒，10秒后数据是否存在？
  * 1、定时过期：redis会扫描所有数据是否过期，如果过期，删除；
    * 节省内存
    * cpu占用高、消耗计算机的性能；
  * 2、定期过期：redis会扫描部分数据是否过期，如果过期，删除；
    * 内存可能会浪费，有部分数据已经过期，但是，没有被扫描到，一直存在；
    * cpu占用也不会太高；
  * 3、惰性过期：redis中数据已经过期，不会立即删除，有人get该数据时，立即判断是否过期，如果过期，删除；
    * 节省cpu
    * 内存浪费

  * redis数据库默认的过期策略是**定期过期+惰性过期。**
  * 弊端：一个数据，有效期已过，没被扫描到，也没有人查询它，会一直常驻内存。

* 缓存淘汰：解决过期数据未被清除的问题。
  * 1、LRU：Least recently used，最近最少使用，按照时间跨度淘汰数据；redis中一直存在的数据都是最新数据；
    * 新数据插入到列表头部；
    * 每当缓存命中（即缓存数据被访问），则将数据移到列表头部；
    * 当列表满的时候，将列表尾部的数据丢弃。
  * 2、LFU：按照数据访问的频率淘汰，redis中一直存在的数据都是热点数据，访问频率高的数据；

* 头条项目是阅读类产品，需要展示最新数据：
  * 淘汰方式是LRU；

* 面试题：mySQL里有2000w数据，redis中只存20w的数据，如何保证redis中的数据都是热点数据？
  * 1、20w数据需要使用的空间大小，配置redis的最大使用内存。
  * 2、淘汰策略LFU，把访问频率低的给淘汰掉；

### 6、缓存模式

* 缓存数据操作的方式：**先更新mysql，再删除redis；**

### 7、缓存问题

* 缓存击穿：客户端故意频繁请求缓存中不存在的数据，造成频繁的直接对mysql的读取；
  * 1、给你不存在的数据，查询mysql没有，**故意缓存无效数据**，-1；
  * 2、过滤请求信息，布隆过滤器；
* 缓存雪崩：缓存数据有效期在一个时间点，大量数据集体失效；
  * 1、多级缓存
  * 2、有效期不固定，基准值30分钟+偏差值（1-到5分钟）
  * 3、加锁或队列；

* 头条项目中应用：
  * 1、缓存击穿通过缓存无效数据解决；
  * 2、缓存雪崩通过有效期不固定解决；























